In [1]:
import csv
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise import evaluate, print_perf
import surprise
from helpers import *
import pandas as pd

In [2]:
ratings = load_data('data_train.csv')
num_items_per_user = np.array((ratings != 0).sum(axis=0)).flatten()
num_users_per_item = np.array((ratings != 0).sum(axis=1).T).flatten()

number of items: 10000, number of users: 1000


In [3]:
_,train, test = split_data(ratings, num_items_per_user, num_users_per_item, 10, 0.1)

the shape of original ratings. (# of row, # of col): (10000, 1000)
the shape of valid ratings. (# of row, # of col): (9990, 999)
Total number of nonzero elements in origial data:1176952
Total number of nonzero elements in train data:1065253
Total number of nonzero elements in test data:111620


<9990x999 sparse matrix of type '<class 'numpy.float64'>'
	with 111620 stored elements in LInked List format>

In [4]:
nnz_row, nnz_col = test.nonzero()
nnz_test = list(zip(nnz_row, nnz_col))
test.shape

(9990, 999)

In [55]:
def svd_surprise(train, test):
    
    train_df = create_data(train)
    reader = surprise.dataset.Reader(rating_scale=(1,5))
    data = Dataset.load_from_df(train_df[['row', 'col', 'rate']], reader)
    data.split(2) 
    
    algo = SVD(n_factors=20)
    algo.train(data.build_full_trainset())
    
    test_df = train_df.copy()
    temp = []
    if test != None:
        test_df = create_data(test)
    for index, row in test_df.iterrows():
        temp.append((int (row['row']), int (row['col']), row['rate']))
    
    predictions = algo.test(temp)
    final_predictions = []
    
    for pred in predictions:
        final_predictions.append((pred[0], pred[1], pred[3]))
    pred_df = pd.DataFrame(preprocess_data(final_predictions, True).todense())
    if test != None:
        nnz_row, nnz_col = test.nonzero()
        nnz_test = list(zip(nnz_row, nnz_col))
        print(compute_mix_error(data=test, prediction=pred_df.as_matrix(), nz=nnz_test))
    return pred_df.as_matrix()

In [50]:
def create_data(dataset):
    dataset_matrix = pd.DataFrame(dataset.todense())
    dataset_df = pd.DataFrame(dataset_matrix.unstack())
    dataset_df = dataset_df.reset_index() 
    dataset_df.rename(columns={'level_0': 'col', 'level_1': 'row', 0: 'rate'}, inplace=True)
    dataset_df['col'] +=1
    dataset_df['row'] +=1
    return dataset_df[dataset_df['rate'] != 0]
    

In [56]:
svd_surprise(ratings,None)

number of items: 10000, number of users: 1000


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  3.39558382,
         3.15292525,  3.02713528],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  3.44099915]])

In [34]:
temp = []
test_matrix = pd.DataFrame(test.todense())
o = test_matrix.unstack()
p = pd.DataFrame(o)
p = p.reset_index()
p['level_0'] +=1
p['level_1'] +=1
p=p[p[0] != 0]
p = p[['level_1', 'level_0', 0]]


In [35]:
predictions = algo.test(temp)
predictions

[Prediction(uid=67, iid=1, r_ui=4.0, est=2.7909401349151941, details={'was_impossible': False}),
 Prediction(uid=108, iid=1, r_ui=3.0, est=3.4262268594070258, details={'was_impossible': False}),
 Prediction(uid=410, iid=1, r_ui=2.0, est=2.8249812285364171, details={'was_impossible': False}),
 Prediction(uid=457, iid=1, r_ui=2.0, est=2.775900163636615, details={'was_impossible': False}),
 Prediction(uid=1337, iid=1, r_ui=3.0, est=2.8580015915486152, details={'was_impossible': False}),
 Prediction(uid=1526, iid=1, r_ui=2.0, est=3.3918474944889923, details={'was_impossible': False}),
 Prediction(uid=1621, iid=1, r_ui=5.0, est=4.0756102244546932, details={'was_impossible': False}),
 Prediction(uid=2426, iid=1, r_ui=5.0, est=2.78563423255622, details={'was_impossible': False}),
 Prediction(uid=2878, iid=1, r_ui=2.0, est=2.81425161146508, details={'was_impossible': False}),
 Prediction(uid=2937, iid=1, r_ui=4.0, est=3.2895035199449558, details={'was_impossible': False}),
 Prediction(uid=3125

In [ ]:
def svd(train, test):
    algo = SVD()
    data = 

In [7]:
def preprocessed(line):
    ids, rating = line.split(',')
    user_id, movie_id = ids.split('_')
    user_id = user_id.replace('r', '')
    movie_id = movie_id.replace('c', '')
    return int(user_id), int(movie_id), int(rating)

f = open('data_train.csv', 'r')
data = f.read().splitlines()[1:]
data = list(map(lambda line: preprocessed(line), data))
data[0]

(44, 1, 4)

In [5]:
ratings_df = pd.DataFrame.from_records(data, columns=['user_id', 'movie_id', 'rating'])
reader = surprise.dataset.Reader(rating_scale=(1,5))

data = Dataset.load_from_df(ratings_df[['user_id', 'movie_id', 'rating']], reader)
data.split(2) 

In [6]:
algo = SVD()

# Evaluate performances of our algorithm on the dataset.
trainset = data.build_full_trainset()
algo.train(trainset)

In [8]:

f = open('sample_submission.csv', 'r')
test = f.read().splitlines()[1:]
test = list(map(lambda line: preprocessed(line), test))

In [9]:
predictions = algo.test(data)

In [11]:
len(predictions)

1176952

In [36]:
final_predictions = []
for pred in predictions:
    final_predictions.append((pred[0], pred[1], pred[3]))

In [37]:
predictions_SVD = preprocess_data(final_predictions, True)

number of items: 9990, number of users: 999


In [38]:
pred_df = pd.DataFrame(predictions_SVD.todense())
pred_df

,0,1,2,3,4,5,6,7,8,9,...,989,990,991,992,993,994,995,996,997,998
0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
6,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
7,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
8,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,2.613313,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
9,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [39]:
compute_mix_error(data=test, prediction=pred_df.as_matrix(), nz=nnz_test)

1.004077988101316